<a href="https://colab.research.google.com/github/momchilgeorgiev/2023DL/blob/main/Testing_this_time_it_works.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Dense, Flatten, Conv2D, \
                             Dropout, Input, MaxPool2D, Rescaling, BatchNormalization, \
                             GlobalAveragePooling2D, Concatenate, Reshape, Input, UpSampling2D

from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential, Model

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass

In [3]:
(ds_train, ds_test), ds_info = tfds.load(
    'cifar100',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

In [4]:
ds_train.element_spec

(TensorSpec(shape=(32, 32, 3), dtype=tf.uint8, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [ ]:
ds_info.features

FeaturesDict({
    'coarse_label': ClassLabel(shape=(), dtype=int64, num_classes=20),
    'id': Text(shape=(), dtype=string),
    'image': Image(shape=(32, 32, 3), dtype=uint8),
    'label': ClassLabel(shape=(), dtype=int64, num_classes=100),
})

### Dataset to numpy:

### Preprocessing #2

In [5]:
def preprocess(image, label):
    image = tf.image.per_image_standardization(image)

    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
    return image, label

In [6]:
AUTO = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 16
NUM_CLASSES = ds_info.features['label'].num_classes

ds_train = ds_train.cache().shuffle(1000).batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTO)

ds_test = ds_test.batch(BATCH_SIZE)
ds_test = ds_test.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [7]:
ds_train.element_spec

(TensorSpec(shape=(None, 32, 32, 3), dtype=tf.uint8, name=None),
 TensorSpec(shape=(None,), dtype=tf.int64, name=None))

In [9]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input as preprocess_input_VGG

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_input_Resnet50

from tensorflow.keras.applications.densenet import DenseNet121

In [257]:
tf.keras.backend.clear_session()

In [10]:
# Load the VGG16 model with ImageNet weights
vgg16 = VGG16(weights='imagenet', include_top=False)

# Load the DenseNet121 model with ImageNet weights
densenet121 = DenseNet121(weights='imagenet', include_top=False)

vgg16.trainable = False
densenet121.trainable = False
# Reshape the output of VGG16 to match the input shape of DenseNet121
# Assuming a common denominator shape for both
vgg16_output = vgg16.output
reshape_layer = Reshape((32, 32, 3))(vgg16_output)

# Add the reshaped output of VGG16 as input to DenseNet121
densenet121_input = densenet121(reshape_layer)

In [11]:
# Add a global average pooling layer to reduce the dimensions
gap_layer = GlobalAveragePooling2D()(densenet121_input)

# Add a fully connected layer with, for example, 256 neurons
fc_layer = Dense(256, activation='relu')(gap_layer)

# Add a final output layer, for example, for a classification task with 10 classes
output_layer = Dense(NUM_CLASSES, activation='softmax')(fc_layer)

# Create a new model with the input of VGG16 and the new output layer
custom_model = Model(inputs=vgg16.input, outputs=output_layer)

In [16]:
vgg16_output = vgg16.get_layer('block5_pool').output

In [17]:
vgg16_model = Model(inputs=vgg16.input, outputs=vgg16_output)

In [29]:
bridge_layer = Conv2D(256, (1, 1), padding='same', activation='relu')(vgg16_model.output)

In [33]:
bridge_layer = Conv2D(3, (1, 1), padding='same', activation='relu')(vgg16_model.output)

# Load DenseNet121 model without the top layers
densenet121 = DenseNet121(weights='imagenet', include_top=False)

# Use the output of the bridge layer as input to DenseNet121
densenet121_output = densenet121(bridge_layer)

In [34]:
final_model = Model(inputs=vgg16_model.input, outputs=densenet121_output)

In [36]:
final_model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [37]:
final_model.fit(ds_train)

ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None

    ValueError: Exception encountered when calling layer 'pool2_pool' (type AveragePooling2D).
    
    Negative dimension size caused by subtracting 2 from 1 for '{{n